In [8]:
##@YZ April 2024
import quimb.tensor as qtn
import quimb as qu
from functions import*
import torch
import cotengra as ctg


In [5]:
def norm_fn(psi):
    # parametrize our tensors as isometric/unitary
    return psi.isometrize(method='cayley')

def loss_fn(psi):
    # compute the total energy, here quimb handles constructing 
    # and contracting all the appropriate lightcones 
    return - abs((psi_tar.H & psi).contract(all, optimize=opti)) ** 2

class TNModel(torch.nn.Module):

    def __init__(self, tn):
        super().__init__()
        # extract the raw arrays and a skeleton of the TN
        params, self.skeleton = qtn.pack(tn)
        # n.b. you might want to do extra processing here to e.g. store each
        # parameter as a reshaped matrix (from left_inds -> right_inds), for 
        # some optimizers, and for some torch parametrizations
        self.torch_params = torch.nn.ParameterDict({
            # torch requires strings as keys
            str(i): torch.nn.Parameter(initial)
            for i, initial in params.items()
        })

    def forward(self):
        # convert back to original int key format
        params = {int(i): p for i, p in self.torch_params.items()}
        # reconstruct the TN with the new parameters
        psi = qtn.unpack(params, self.skeleton)
        # isometrize and then return the energy
        return loss_fn(norm_fn(psi))

#the following is an optimizer for speeding up tensor network contractions
opti = ctg.ReusableHyperOptimizer(
    progbar=True,
    methods=['greedy'],
    reconf_opts={},
    max_repeats=36,
    optlib='random',
    # directory=  # set this for persistent cache
)

In [6]:
L = 14
in_depth = L #RQC depth
psi_2 = qmps_f(L, in_depth=in_depth, n_Qbit=L-1, qmps_structure="brickwall", canon="left")
depth_initial,depth_final,depth_step = 1,L//2+1,1 # PQC depth
peak_wights = []

# here we use a sequential optimization scheme; namely we gradually add PQC layers and use the previous optimization results as an intialization 
for depth in range(depth_initial,depth_final,depth_step):
    psi_pqc = qmps_f(L, in_depth= depth, n_Qbit=L-1, qmps_structure="brickwall", canon="left",start_layer = (in_depth)%2,rand = True)
    psi = psi_pqc.tensors[L]
    
    # here we seperate the 'all-zero state' to the PQC circuit as we don't want to optimize over that
    for i in range (L+1,len(psi_pqc.tensors)):
        psi = psi&psi_pqc.tensors[i]
    if depth != depth_initial:
        psi_c = psi.copy()
        psi = load_para(psi_c, dictionary)
    
    psi_tar = psi_2.copy()
    for i in range (L):
        psi_tar = psi_tar&psi_pqc.tensors[i] 
    psi.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.complex128))
    psi_tar.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.complex128))
    psi_2.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.complex128))
    
    model = TNModel(psi)
    model()
    import warnings
    from torch import optim
    with warnings.catch_warnings():
        warnings.filterwarnings(
            action='ignore',
            message='.*trace might not generalize.*',
        )
        model = torch.jit.trace_module(model, {"forward": []})
        
    import torch_optimizer
    import tqdm
    
    optimizer = optim.Adam(model.parameters(), lr=.001)
    
    
    its = 5000
    pbar = tqdm.tqdm(range(its),disable=False)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=300, gamma=0.5)
    previous_loss = torch.inf
    for step in pbar:
        show_progress_bar=True
        optimizer.zero_grad()
        loss = model()
        loss.backward()
        def closure():
            return loss
        optimizer.step()
        pbar.set_description(f"{loss}")
        progress_bar_refresh_rate=0
        if step > 100 and torch.abs(previous_loss - loss) < 1e-10:
            print("Early stopping loss difference is smaller than 1e-10")
            break
        previous_loss = loss.clone()
    dictionary = save_para(psi)
    peak_wights.append(loss_fn(norm_fn(psi)))

1


  0%|                                                    | 0/36 [00:00<?, ?it/s]/opt/anaconda3/envs/py312/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(
/opt/anaconda3/envs/py312/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(
/opt/anaconda3/envs/py312/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(
/opt/anaconda3/envs/py312/lib/python3.11/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


Early stopping loss difference is smaller than 1e-10
2


-0.03926272960715059:  50%|████████▌        | 2507/5000 [00:31<00:31, 78.46it/s]
/var/folders/xq/n67s1gcd4h3_ddhyjq8gjt_9pymrz4/T/ipykernel_5134/2028719282.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  psi.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.complex128))
/var/folders/xq/n67s1gcd4h3_ddhyjq8gjt_9pymrz4/T/ipykernel_5134/2028719282.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  psi_tar.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.complex128))
/var/folders/xq/n67s1gcd4h3_ddhyjq8gjt_9pymrz4/T/ipykernel_5134/2028719282.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach

Early stopping loss difference is smaller than 1e-10
3


...

-0.1622811700338676:  74%|█████████████▎    | 3704/5000 [00:58<00:20, 63.14it/s]
/var/folders/xq/n67s1gcd4h3_ddhyjq8gjt_9pymrz4/T/ipykernel_5134/2028719282.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  psi.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.complex128))
/var/folders/xq/n67s1gcd4h3_ddhyjq8gjt_9pymrz4/T/ipykernel_5134/2028719282.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  psi_tar.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.complex128))
/var/folders/xq/n67s1gcd4h3_ddhyjq8gjt_9pymrz4/T/ipykernel_5134/2028719282.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach

Early stopping loss difference is smaller than 1e-10
6


-0.21125544481616435: 100%|█████████████████| 5000/5000 [02:16<00:00, 36.58it/s]
/var/folders/xq/n67s1gcd4h3_ddhyjq8gjt_9pymrz4/T/ipykernel_5134/2028719282.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  psi.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.complex128))
/var/folders/xq/n67s1gcd4h3_ddhyjq8gjt_9pymrz4/T/ipykernel_5134/2028719282.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  psi_tar.apply_to_arrays(lambda x: torch.tensor(x, dtype=torch.complex128))
/var/folders/xq/n67s1gcd4h3_ddhyjq8gjt_9pymrz4/T/ipykernel_5134/2028719282.py:23: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach

7


-0.2410062028134099: 100%|██████████████████| 5000/5000 [01:53<00:00, 44.09it/s]


In [7]:
peak_wights

[tensor(-0.0132, dtype=torch.float64),
 tensor(-0.0393, dtype=torch.float64),
 tensor(-0.0641, dtype=torch.float64),
 tensor(-0.1265, dtype=torch.float64),
 tensor(-0.1623, dtype=torch.float64),
 tensor(-0.2113, dtype=torch.float64),
 tensor(-0.2410, dtype=torch.float64)]